# Car Damage Detection

### Imports

In [2]:
import os
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

import torch.nn as nn
from torchvision.models import resnet18

import torch.optim as optim

### Data Preparation

- Load data
- Label Encoding

In [3]:
### Paths to dataset
train_dir = 'dataset/training/'
val_dir = 'dataset/validation/'

### Transformations for loading images
### Resizing images to 224X224
### Converting images to tensors
base_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])

### Load datasets
train_dataset = datasets.ImageFolder(train_dir, transform=base_transform)
val_dataset = datasets.ImageFolder(val_dir, transform=base_transform)

### Create data loaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

### Label encoding (0: damaged, 1: whole)
class_names = train_dataset.classes
print(f'Classes: {class_names}')

Classes: ['00-damage', '01-whole']


### Data Preprocessing

- Resize Images
- Normalize
- Augmentation

In [4]:
# Define training transformations with augmentation
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),  # Random horizontal flip
    transforms.RandomRotation(10),      # Random rotation up to 10 degrees
    transforms.ColorJitter(brightness=0.2, contrast=0.2),  # Random brightness/contrast
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],  # Standard normalization for ResNet
                         std=[0.229, 0.224, 0.225])
])

# Update train dataset with augmented transforms
train_dataset.transform = train_transform

# Validation transform (no augmentation, only normalization)
val_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

# Update validation dataset
val_dataset.transform = val_transform

### Model: ResNet18 (Transfer Learning)

- Load pre-trained ResNet18.
- Replace final layer with binary output.

In [5]:
# Load pre-trained ResNet18
model = resnet18(pretrained=True)

# Modify the final layer for binary classification (2 classes: damaged, whole)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 2)

# Move model to GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

d:\Projects\AI\Car-Damage-Detection\.venv\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
d:\Projects\AI\Car-Damage-Detection\.venv\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


### Training
- Train the model with a optimizer and loss function.


In [6]:
# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        
        # Zero the gradients
        optimizer.zero_grad()
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward pass and optimize
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item() * images.size(0)
    
    # Print epoch loss
    epoch_loss = running_loss / len(train_dataset)
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}')

# Save the trained model
torch.save(model.state_dict(), 'car_damage_model.pth')

Epoch [1/10], Loss: 0.3912
Epoch [2/10], Loss: 0.2736
Epoch [3/10], Loss: 0.2256
Epoch [4/10], Loss: 0.1895
Epoch [5/10], Loss: 0.1911
Epoch [6/10], Loss: 0.1814
Epoch [7/10], Loss: 0.1592
Epoch [8/10], Loss: 0.1391
Epoch [9/10], Loss: 0.1099
Epoch [10/10], Loss: 0.1363


## 5. Evaluation
- Evaluate the model on the validation set.

In [7]:
# Evaluate the model
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in val_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f'Validation Accuracy: {accuracy:.2f}%')

Validation Accuracy: 74.13%


## 6. Test

In [25]:
from PIL import Image

# Define the same transformation as validation
test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

# Function to test a single image
def classify_image(image_path):
    # Load the image
    image = Image.open(image_path).convert('RGB')
    
    # Apply transformations
    image = test_transform(image)
    image = image.unsqueeze(0)  # Add batch dimension
    
    # Move image to device
    image = image.to(device)
    
    # Set model to evaluation mode
    model.eval()
    with torch.no_grad():
        output = model(image)
        _, predicted = torch.max(output, 1)
        
    # Get the predicted class
    predicted_class = class_names[predicted.item()]
    return predicted_class

# Example usage: Replace 'path/to/your/image.jpg' with the actual image path
image_path = 'D:\Projects\AI\Car-Damage-Detection\dataset\\testing\\00-damage\\0009.jpg'
result = classify_image(image_path)
print(f'The image is classified as: {result}')

The image is classified as: 01-whole
